Put signs in the camera views

In [ ]:
from gtsrb_loader.get_folderpath import get_folderpath
from gtsrb_loader.load_data import load_data

from skimage import data
from skimage.transform import resize
import numpy as np
import pandas as pd
import os
import fnmatch
import random
from PIL import Image
# from cs231n
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from __future__ import print_function

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
#Settings
#Cloudbuchstabe
cloud='Y:\\Zero_Class_color\\00043'
cloud_dest='Y:\\Images'

In [ ]:
# sourepictures
dir_list =[]
csv_filelist = []
path = get_folderpath(subset='train', original_images=False)
ppm_filelist = []

for root, dirs, files in os.walk(path):
    dir_list += [os.path.join(root, directory)for directory in dirs if directory not in dir_list ]
    csv_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.csv')]
    ppm_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.ppm')]


In [ ]:
dirs_dest=[]
for dirs in dir_list:
    dirs_dest += [dirs.split('Images\\')[1]]
dirs_dest

In [ ]:
# destination, die Fahrsteckenbilder ohne Schilder
dest_list=[]
#pathdest = #phoenix 0Klasse
for root, dirs, files in os.walk(cloud):
    dest_list += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.ppm')]

In [ ]:
pic_array_list=[]
for dirs_master in dir_list:
    #pic_array_list.append([pic for pic in ppm_filelist if fnmatch.fnmatch(pic, dirs)])
    #for pic in ppm_filelist:
     #   if fnmatch.fnmatch(pic, dirs):
      #      print('true')
    ppm_filelist2 = []
    for root, dirs, files in os.walk(dirs_master):
        ppm_filelist2 += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.ppm')]
    pic_array_list.append(ppm_filelist2)

In [ ]:
#destination picutres
os.path.split(ppm_filelist[1])[1]

In [ ]:
def transform_csv(df: pd.DataFrame) -> pd.DataFrame:
    columns=['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId']
    matrix = df.as_matrix(columns=columns)
    matrix.shape
    matrix2 = np.zeros(matrix.shape)
    for i, row in enumerate(matrix):
        matrix2[i] =[row[0],row[1],row[0]-row[2],row[3],row[0]-row[4],row[5], row[6]]
    matrix2 = matrix2.astype(int)
    df2 = pd.DataFrame(matrix2, columns=columns)
    df2['Filename'] = df['Filename']
    df2 = df2[['Filename'] + columns]
    return df2

In [ ]:
pixels = 16
n_dest=3600
new_path = path.split('\GTSRB_64x64')[1]
new_path=cloud_dest+'\GTSRB_64x64_PiP'+new_path
random.seed()
columns=['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId']
idx_dest=0
    #for idx, dirpath in enumerate(dir_list):
     #   filepath=csv_filelist[idx]
      #  if not fnmatch.fnmatch(filepath, '*GT-final_test.test.csv'): # because this csv has no classes
       #     df = pd.read_csv(filepath, sep=';')
        #files ist das Array aller Bilder im Ordnert index
        #Für alle Ordner
for idx, pic_array in enumerate(pic_array_list):
    #für alles Bilder im Ordner
    if idx !=32:
        continue
    filepath=csv_filelist[idx]
    if not fnmatch.fnmatch(filepath, '*GT-final_test.test.csv'): # because this csv has no classes
        df = pd.read_csv(filepath, sep=';')   
    matrix = df.as_matrix(columns=columns)
    for idx_obj,pics in enumerate(pic_array):
        dest=np.asarray(plt.imread(dest_list[idx_dest]))
        pic = plt.imread(pics)
        resized = resize(pic, (pixels, pixels), mode='edge')
        X1=int(df.iloc[idx_obj]['Roi.X1']/4) #Abrunden
        X2=int(df.iloc[idx_obj]['Roi.X2']/4) + (df.iloc[idx_obj]['Roi.X2'] % 4 > 0) #Aufrunden
        Y1=int(df.iloc[idx_obj]['Roi.Y1']/4) #Abrunden
        Y2=int(df.iloc[idx_obj]['Roi.Y2']/4) + (df.iloc[idx_obj]['Roi.Y2'] % 4 > 0) #Aufrunden
        distX=X2-X1
        distY=Y2-Y1
        np.asarray
        im = np.uint8(resized * 255)
        inputpointX = random.randint(24,48) #Zielort im Zielbild
        inputpointY = random.randint(24,48) #Zielort im Zielbild
        matrix[idx_obj] =[64,64,inputpointX,inputpointY,inputpointX+distX,inputpointY+distY, df.iloc[idx_obj]['ClassId']]
        cropped = im[X1:X2, Y1:Y2]
        dest[inputpointX:inputpointX+distX,inputpointY:inputpointY+distY]=cropped
        im_new=Image.fromarray(dest)
        idx_dest=(idx_dest+1)%n_dest
        if not os.path.exists(new_path+'\\'+dirs_dest[idx]):
            os.makedirs(new_path+'\\'+dirs_dest[idx]) 
        im_new.save(new_path+'\\'+dirs_dest[idx]+'\\'+os.path.split(pics)[1])
    df2 = pd.DataFrame(matrix, columns=columns)
    df2['Filename'] = df['Filename']
    df2 = df2[['Filename'] + columns]
    n='GT-'+dirs_dest[idx]+'.csv'
    df2.to_csv(new_path+'\\'+dirs_dest[idx]+'\\'+n, sep=';', index=False)


In [ ]:
path = get_folderpath(subset='test', original_images=False)

    

In [ ]:
#dir_list

        #if not fnmatch.fnmatch(csv_filelist[1], '*GT-final_test.test.csv'): # because this csv has no classes
pd.read_csv(csv_filelist[1], sep=';')

In [ ]:
path = get_folderpath(subset='train', original_images=False)
ppm_filelist = []
csv_filelist = []
for root, dirs, files in os.walk(path):
    ppm_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.ppm')]
    csv_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.csv')]

In [ ]:
pic_array_list[0]

In [ ]:
for idx, dirpath in enumerate(dir_list):
    for idx2, root in enumerate(os.walk(dirpath)):
        print(root)
        print('\n')

In [ ]:
    for filepath in csv_filelist:
        if not fnmatch.fnmatch(filepath, '*GT-final_test.test.csv'): # because this csv has no classes
            df = pd.read_csv(filepath, sep=';')
            #df2 = _transform_csv(df)

In [ ]:
#pd.read_csv(csv_filelist[1], sep=';')
#root.size
#for lists in pic_array_list:
#  print(type(lists))
df

df.iloc[0]['Roi.X1']
df.iloc[0]['Roi.X2']
df.iloc[0]['Roi.Y1']
df.iloc[0]['Roi.Y2']